In [7]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
import matplotlib 
import os
import yaml

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import sys, os
root = os.path.join(os.getcwd(), '../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Distgen 
---

In [9]:
from distgen import Generator

DISTGEN_INPUT_FILE = os.path.join(root,'templates/tesla_9cell_cavity/distgen.in.yaml')

G = Generator(DISTGEN_INPUT_FILE)
G.run()

yaml.dump(yaml.safe_load(DISTGEN_FILE_STR), open(DISTGEN_FILE,'w'))


# GPT input file:

In [6]:
GPT_INPUT_FILE = os.path.join(root,'templates/tesla_9cell_cavity/gpt.in')

# Run GPT without phasing
---

In [20]:
from gpt.gpt_distgen import run_gpt_with_distgen

settings={'n_particle':200, 
          'Ntout':500,
          'field_scale':0}

G = run_gpt_with_distgen(settings,
                         gpt_input_file=GPT_INPUT_FILE,
                         distgen_input_file=DISTGEN_INPUT_FILE,
                         verbose=True,
                         auto_phase=True)

print(G)

Run GPT with Distgen:
Replacing param n_particle with value 200
Ntout is in gpt
field_scale is in gpt
Linking particle files, distgen output will point to -> "gpt.particles.gdf" in working directory.

Distgen >------

Distribution format: gpt
Output file: /var/folders/t8/f3n1sdnn1jg0wfbpqfm1hsh80000gn/T/tmpwcgw36el/gpt.particles.gdf

Creating beam distribution....
   Beam starting from: time
   Total charge: 1 nC.
   Number of macroparticles: 200.
   Assuming cylindrical symmetry...
   r distribution: radial Gaussian
   theta distribution:       min_theta = 0 rad, max_theta = 6.28319 rad
   z distribution: Gaussian
      avg_z = 0 mm, sigma_z = 0.100 mm
   Shifting avg_x = -1.27662E-05 mm -> 0 mm
   Scaling sigma_x = 0.000982028 mm -> 0.001 mm
   Shifting avg_y = -2.88062E-06 mm -> 0 mm
   Scaling sigma_y = 0.000987073 mm -> 0.001 mm
   Shifting avg_z = -0.00197851 mm -> 0 mm
   Scaling sigma_z = 0.0976921 mm -> 0.1 mm
   Time start: fixing all particle time values to start time: 0 s.
